In [2]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [3]:
validatedf = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")

df= df.drop(['latitude','longitude', 'id', 'carNumber', 'date', 'time', 'race'], axis=1)
validatedf = validatedf.drop(['latitude','longitude', 'id', 'carNumber', 'date', 'time', 'race'], axis=1)

In [4]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer

# Assuming 'circuit', 'driver', and 'constructor' are categorical columns
df = pd.get_dummies(df, columns=['circuit', 'driver', 'constructor'])

# Extract features (X) and target variable (y)
X = df.drop(columns=['lapTime_ms'])
y = df['lapTime_ms']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features using Z-score normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the Gradient Boosting model
gb_model = GradientBoostingRegressor()

# Set up hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Define the scoring metric for GridSearchCV
scoring_metric = make_scorer(mean_squared_error, greater_is_better=False)

# Create GridSearchCV instance
grid_search_gb = GridSearchCV(gb_model, param_grid=param_grid, scoring=scoring_metric, cv=5)

# Fit the model using GridSearchCV
grid_search_gb.fit(X_train_scaled, y_train)

# Print the best hyperparameter combination
best_params_gb = grid_search_gb.best_params_
print("Best Hyperparameter Combination for Gradient Boosting:")
print(best_params_gb)

# Evaluate the best model on the test set
mse_gb = grid_search_gb.score(X_test_scaled, y_test)
print(f'Mean Squared Error on Test Set (Gradient Boosting): {mse_gb}')

# Make predictions on new data (e.g., validatedf)
validatedf_encoded = pd.get_dummies(validatedf, columns=['circuit', 'driver', 'constructor'])
validatedf_encoded = validatedf_encoded.reindex(columns=X_train.columns, fill_value=0)
X_valid_scaled_gb = scaler.transform(validatedf_encoded)
predicted_laptimes_valid_gb = grid_search_gb.predict(X_valid_scaled_gb)


Best Hyperparameter Combination for Gradient Boosting:
{'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 50}
Mean Squared Error on Test Set (Gradient Boosting): -993663161.5922345


In [5]:
predicted_laptimes_valid_gb

array([102672.77988322,  70016.3608095 ,  97469.18626262, ...,
       110846.7657032 , 136503.59152915,  86905.39511925])